Alleles_frequency_table.zip can be unzipped to a tab-separated text file that shows all reads and alignments to references. The first column shows the aligned sequence of the sequenced read. The second column shows the aligned sequence of the reference sequence. Gaps in each of these columns represent insertions and deletions. The next column 'Reference_Name' shows the name of the reference that the read aligned to. The fourth column, 'Read_Status' shows whether the read was modified or unmodified. The fifth through seventh columns ('n_deleted', 'n_inserted', 'n_substituted') show the number of bases deleted, inserted, and substituted as compared to the reference sequence. The eighth column shows the number of reads having that sequence, and the ninth column shows the percentage of all reads having that sequence.

In [1]:
# #features to add:
# Distance from TSS - get relative position of mutation in the guide site - done. Add distance from cut site metric. - done Then calculate distance from Araport TSS - done
# for this: first create a bed file for all of the mutations (relative to whole Arabidopsis genome). Then do bedtools merge or intersect (or bedtools coverage (../data_sorting/./TFBS_coverage.sh)) with the mapped motif bed file (all TFBSs for all genes). Record each TFBS that overlaps the mutation
# Overlapping TFBSs - subnet
# work and all TFs

# Include secondary mutations in case both deletion and substitution for example - done
# Plant ID
# How many biallelic or homozygous? How many wildtype?
# More than 2 alleles for a gene - record alleles until 80% of reads accounted for
# Prioritise homozygous or biallelic
# How many plants had mutations? How many guides produced mutations in each gene?
#check window around cut site - at the moment I am including mutations 20bp either side, maybe cut the alignments down to 7bp either side before comparing them with find_indels_substitutions()


In [2]:
#use env pybedtools
import pandas as pd
import numpy as np
import io
from pybedtools import BedTool
from collections import defaultdict
#chunks
from more_itertools import sliced
import multiprocessing as mp

import time
import os
#import dask.dataframe as dd

In [3]:
def read_in_files(mutations_file,mapped_motifs,guide_pairs):
    """read in the files"""
    #read in mapped motifs bed file
    mapped_motifs = pd.read_table(mapped_motifs_bed, sep="\t", header=None)
    if len(mapped_motifs.columns) == 24:
        cols = [
            "chr",
            "start",
            "stop",
            "promoter_AGI",
            "dot1",
            "strand",
            "source",
            "type",
            "dot2",
            "attributes",
            "motif_chr",
            "motif_start",
            "motif_stop",
            "name_rep",
            "score",
            "motif_strand",
            "promoter_AGI2",
            "p-value",
            "q-value",
            "matched_sequence",
            "TF_name",
            "TF_family",
            "TF_AGI",
            "bp_overlap",
        ]
        mapped_motifs.columns = cols
        # filter columns
        mapped_motifs = mapped_motifs[
            [
                "motif_chr",
                "motif_start",
                "motif_stop",
                "name_rep",
                "score",
                "motif_strand",
                "promoter_AGI2",
                "p-value",
                "q-value",
                "matched_sequence",
                "TF_name",
                "TF_family",
                "TF_AGI",
            ]
        ]
        #rename columns
        cols = [
            "motif_chr",
            "motif_start",
            "motif_stop",
            "name_rep",
            "score",
            "motif_strand",
            "promoter_AGI",
            "p-value",
            "q-value",
            "matched_sequence",
            "TF_name",
            "TF_family",
            "TF_AGI",
        ]
        mapped_motifs.columns = cols

    elif len(mapped_motifs.columns) == 13:
        cols = [
            "motif_chr",
            "motif_start",
            "motif_stop",
            "name_rep",
            "score",
            "motif_strand",
            "promoter_AGI",
            "p-value",
            "q-value",
            "matched_sequence",
            "TF_name",
            "TF_family",
            "TF_AGI",
        ]
        mapped_motifs.columns = cols

    elif len(mapped_motifs.columns) == 17:
        cols = [
            "motif_chr",
            "motif_start",
            "motif_stop",
            "name_rep",
            "score",
            "motif_strand",
            "promoter_AGI",
            "p-value",
            "q-value",
            "matched_sequence",
            "TF_name",
            "TF_family",
            "TF_AGI",
            "chr_openchrom",
            "start_openchrom",
            "stop_openchrom",
            "bp_overlap",
        ]
        mapped_motifs.columns = cols

    mutations_df = pd.read_table(mutations_file,sep='\t',header=0)
    guide_pairs_df = pd.read_csv(guide_pairs,header=0)
    #only keep first 2 columns
    guide_cols = ['guide1','guide2']
    guide_pairs_df = guide_pairs_df[guide_cols]
    return mutations_df,mapped_motifs,guide_pairs_df


In [4]:
def merge_bedfiles(bedfile, mapped_motifs_bed, output_buffer):
    """perform bedtools intersect on the two dfs"""
    df = BedTool(bedfile)
    motifs = BedTool(mapped_motifs_bed)
    # -wao =Write the original A and B entries plus the number of base pairs of overlap between the two features.
    # However, A features w/o overlap are also reported with a NULL B feature and overlap = 0
    intersect = df.intersect(motifs, wao=True)
    # Write to output_file
    # Each line in the file contains bed entry a and bed entry b that it overlaps plus the number of bp in the overlap so 19 columns
    output_buffer.write(str(intersect))
    #go back to beginning of buffer
    output_buffer.seek(0)
    mapped_motifs_bed.seek(0)
    return output_buffer


In [23]:
def find_overlapping_TFBSs(mutations_df_chunk,mapped_motifs_bed,mapped_motifs_bed_columns):
    """function to find any overlapping TFBSs from FIMO mapped motif file"""
    #go back to start of buffer
    mapped_motifs_bed.seek(0)
    #find mutations within
    for mutations_df_index,mutations_df_row in mutations_df_chunk.iterrows():
            
        if mutations_df_row.mutation_type == 'None':
            pass
        else:
            #create temporary df in bed format
            
            cols = ["chr",
            "start",
            "stop",
            "mutation_type",'mutation_count']
            temp_df = pd.DataFrame(columns=cols)
            chr = mutations_df_row.chr
            #if not NaN
            if pd.notna(mutations_df_row.insertion_genomic_positions) and mutations_df_row.insertion_genomic_positions != 'nan':
                #print(mutations_df_row)
                #print("Index:", index)
                #print(row.insertion_genomic_positions)
                #convert genomic positions from string to list
                # Convert string to list if more than one
                insertion_genomic_positions = mutations_df_row.insertion_genomic_positions.strip('][').split(', ')
                #count which mutation number currently on to be added to the temporary bed file
                count = 0
                for gen_pos in insertion_genomic_positions:
                    count += 1
                    #get index
                    # if len(insertion_genomic_positions) > 1:
                    #     index = insertion_genomic_positions.index(gen_pos)
                    # else:
                    #     index = 'nan'
                    start = int(gen_pos)
                    stop = start + 1
                    mutation_type = "insertion"
                    #add to temp_df
                    temp_list = [chr,start,stop,mutation_type,count]
                    temp_df.loc[len(temp_df)] = temp_list
            if pd.notna(mutations_df_row.deletion_genomic_positions) and mutations_df_row.deletion_genomic_positions != 'nan':
                # Convert string to list
                deletion_genomic_positions = mutations_df_row.deletion_genomic_positions.strip('][').split(', ')
                #count which mutation number currently on to be added to the temporary bed file
                count = 0
                for gen_pos in deletion_genomic_positions:
                    count += 1
                    #get index
                    # if len(deletion_genomic_positions) > 1:
                    #     index = deletion_genomic_positions.index(gen_pos)
                    # else:
                    #     index = 'nan'
                    start = int(gen_pos)
                    stop = start + 1
                    mutation_type = "deletion"
                    #add to temp_df
                    temp_list = [chr,start,stop,mutation_type,count]
                    temp_df.loc[len(temp_df)] = temp_list
            if pd.notna(mutations_df_row.substitution_genomic_positions) and mutations_df_row.substitution_genomic_positions != 'nan':
                # Convert string to list
                substitution_genomic_positions = mutations_df_row.substitution_genomic_positions.strip('][').split(', ')
                #count which mutation number currently on to be added to the temporary bed file
                count = 0
                for gen_pos in substitution_genomic_positions:
                    count += 1
                    #get index
                    # if len(substitution_genomic_positions) > 1:
                    #     index = substitution_genomic_positions.index(gen_pos)
                    # else:
                    #     index = 'nan'
                    start = int(gen_pos)
                    stop = start + 1
                    mutation_type = "substitution"
                    #add to temp_df
                    temp_list = [chr,start,stop,mutation_type,count]
                    temp_df.loc[len(temp_df)] = temp_list
            #now do bedtools intersect to find which TFBSs overlap with which mutations
            #sort by chr then start
            temp_df = temp_df.sort_values(["chr", "start"]).reset_index(drop=True)
            # write to buffer
            temp_df_buffer = io.StringIO()
            temp_df.to_csv(temp_df_buffer, sep="\t", index=False, header=None)
            temp_df_buffer.seek(0)
            
            output_buffer = io.StringIO()
        
            output_buffer = merge_bedfiles(temp_df_buffer, mapped_motifs_bed, output_buffer)
            
            #remove temp_df_buffer stream


            #read in output buffer as df
            output_df = pd.read_table(output_buffer, sep='\t')
            
            #get column names and rename columns
            output_df_cols = cols+mapped_motifs_bed_columns+['bp_overlap']
            output_df.columns = output_df_cols
            #for each mutation type get list of overlapping TFBSs. Add these to a dictionary column in the mutations_df_chunk
            #create defaultdicts with lists as values so that non-existing keys can be added to in one go
            insertion_overlapping_TFBS_family = defaultdict(list)
            insertion_overlapping_TFBS_AGI = defaultdict(list)
            #insertion_overlapping_TFBS_total = defaultdict(list)
            deletion_overlapping_TFBS_family = defaultdict(list)
            deletion_overlapping_TFBS_AGI = defaultdict(list)
            #deletion_overlapping_TFBS_total = defaultdict(list)
            substitution_overlapping_TFBS_family = defaultdict(list)
            substitution_overlapping_TFBS_AGI = defaultdict(list)
            #substitution_overlapping_TFBS_total = defaultdict(list)
            

            for index,row in output_df.iterrows():
                # empty number that will increase for each insertion
               # insertion_overlapping_TFBS_count = int()
               #if mutation and has a 1bp overlap
               #if no TF_AGI then pass
                if row.TF_AGI == '.':
                    pass
                elif row.mutation_type == 'insertion' and row.bp_overlap > 0:
                    #add insertion TFBS family information to dictionary for the correct mutation number
                    #print(row)
                    insertion_overlapping_TFBS_family[f'insertion{row.mutation_count}'] += [row.TF_family]
                    #add insertion TFBS AGI information to dictionary for the correct mutation number
                    insertion_overlapping_TFBS_AGI[f'insertion{row.mutation_count}'] += [row.TF_AGI]
                    #add total number of TFBSs overlapping each insertion
                 #   insertion_overlapping_TFBS_total[f'insertion{row.mutation_count}'] += 1
                elif row.mutation_type == 'deletion' and row.bp_overlap > 0:
                    #add deletion TFBS family information to dictionary for the correct mutation number
                    deletion_overlapping_TFBS_family[f'deletion{row.mutation_count}'] += [row.TF_family]
                    #add deletion TFBS AGI information to dictionary for the correct mutation number
                    deletion_overlapping_TFBS_AGI[f'deletion{row.mutation_count}'] += [row.TF_AGI]
                    #add total number of TFBSs overlapping each insertion
                   # deletion_overlapping_TFBS_total[f'deletion{row.mutation_count}'] += 1
                elif row.mutation_type == 'substitution' and row.bp_overlap > 0:
                    #add substitution TFBS family information to dictionary for the correct mutation number
                    substitution_overlapping_TFBS_family[f'substitution{row.mutation_count}'] += [row.TF_family]
                    #add substitution TFBS AGI information to dictionary for the correct mutation number
                    substitution_overlapping_TFBS_AGI[f'substitution{row.mutation_count}'] += [row.TF_AGI]
                    #add total number of TFBSs overlapping each insertion
                 #   substitution_overlapping_TFBS_total[f'substitution{row.mutation_count}'] += 1
            # #calculate total unique TFBS for each insertion, deletion and subsitution
            # insertion_overlapping_TFBS_total_unique = []
            # for insertion,AGI in insertion_overlapping_TFBS_AGI.items():
            #     insertion_overlapping_TFBS_total_unique += np.unique(AGI).astype(list)
            # deletion_overlapping_TFBS_total_unique = []
            # for deletion,AGI in deletion_overlapping_TFBS_AGI.items():
            #     print(f'AGIunique={np.unique(AGI).astype(list)}')
            #     print(f'total={deletion_overlapping_TFBS_total_unique}')
            #     print(f'AGI={AGI}')
            #     deletion_overlapping_TFBS_total_unique += np.unique(AGI).astype(list)
            #     print(f'newtotal={deletion_overlapping_TFBS_total_unique}')
            # substitution_overlapping_TFBS_total_unique = []
            # for substitution,AGI in substitution_overlapping_TFBS_AGI.items():
            #     substitution_overlapping_TFBS_total_unique += np.unique(AGI).astype(list)
                
            #add values to mutations_df row
            #print(f'insertion_overlapping_TFBS_family={dict(insertion_overlapping_TFBS_family)}')
            #print(mutations_df_row)
            #first make overlapping TFBS families and AGIs unique
            insertion_overlapping_TFBS_family = dict(insertion_overlapping_TFBS_family)
            insertion_overlapping_TFBS_AGI = dict(insertion_overlapping_TFBS_AGI)
            deletion_overlapping_TFBS_family = dict(deletion_overlapping_TFBS_family)
            deletion_overlapping_TFBS_AGI = dict(deletion_overlapping_TFBS_AGI)
            substitution_overlapping_TFBS_family = dict(substitution_overlapping_TFBS_family)
            substitution_overlapping_TFBS_AGI = dict(substitution_overlapping_TFBS_AGI)
            #if empty dictionary, change to nan
            if insertion_overlapping_TFBS_family == {}:
                insertion_overlapping_TFBS_family = np.nan
            else:
                #keep only unique TFBS families
                for k,v in insertion_overlapping_TFBS_family.items():
                    #print(np.unique(v).astype(list))
                    insertion_overlapping_TFBS_family[k] = np.unique(v).tolist()

            if insertion_overlapping_TFBS_AGI == {}:
                insertion_overlapping_TFBS_AGI = np.nan
            else:
                #keep only unique TFBS AGIs
                for k,v in insertion_overlapping_TFBS_AGI.items():
                    #print(np.unique(v).astype(list))
                    insertion_overlapping_TFBS_AGI[k] = np.unique(v).tolist()

            if deletion_overlapping_TFBS_family == {}:
                deletion_overlapping_TFBS_family = 'nan'
            else:
                #keep only unique TFBS families
                for k,v in deletion_overlapping_TFBS_family.items():
                    #print(np.unique(v).astype(list))
                    deletion_overlapping_TFBS_family[k] = np.unique(v).tolist()

            if deletion_overlapping_TFBS_AGI == {}:
                deletion_overlapping_TFBS_AGI = np.nan
            else:
                #keep only unique TFBS AGIs
                for k,v in deletion_overlapping_TFBS_AGI.items():
                    #print(np.unique(v).astype(list))
                    deletion_overlapping_TFBS_AGI[k] = np.unique(v).tolist()
            
            if substitution_overlapping_TFBS_family == {}:
                substitution_overlapping_TFBS_family = np.nan
            else:
                #keep only unique TFBS families
                for k,v in substitution_overlapping_TFBS_family.items():
                    #print(np.unique(v).astype(list))
                    substitution_overlapping_TFBS_family[k] = np.unique(v).tolist()
                    
            if substitution_overlapping_TFBS_AGI == {}:
                substitution_overlapping_TFBS_AGI = np.nan
            else:
                #keep only unique TFBS AGIs
                for k,v in substitution_overlapping_TFBS_AGI.items():
                    #print(np.unique(v).astype(list))
                    substitution_overlapping_TFBS_AGI[k] = np.unique(v).tolist()





            mutations_df_chunk.loc[mutations_df_index, "insertion_overlapping_TFBS_family"] = str(insertion_overlapping_TFBS_family)
            mutations_df_chunk.loc[mutations_df_index, "insertion_overlapping_TFBS_AGI"]= str(insertion_overlapping_TFBS_AGI)
            mutations_df_chunk.loc[mutations_df_index, "deletion_overlapping_TFBS_family"]= str(deletion_overlapping_TFBS_family)
            mutations_df_chunk.loc[mutations_df_index, "deletion_overlapping_TFBS_AGI"]= str(deletion_overlapping_TFBS_AGI)
            mutations_df_chunk.loc[mutations_df_index, "substitution_overlapping_TFBS_family"]= str(substitution_overlapping_TFBS_family)
            mutations_df_chunk.loc[mutations_df_index, "substitution_overlapping_TFBS_AGI"]= str(substitution_overlapping_TFBS_AGI)
            # row.insertion_overlapping_TFBS_total_unique= insertion_overlapping_TFBS_total_unique
            # row.deletion_overlapping_TFBS_total_unique= deletion_overlapping_TFBS_total_unique
            # row.substitution_overlapping_TFBS_total_unique= substitution_overlapping_TFBS_total_unique

            #go back to start of buffer
            mapped_motifs_bed.seek(0)

    return mutations_df_chunk

In [24]:
# def find_overlapping_TFBSs(mutations_df_chunk,mapped_motifs_bed,mapped_motifs_bed_columns):
#     """function to find any overlapping TFBSs from FIMO mapped motif file"""
#     #go back to start of buffer
#     mapped_motifs_bed.seek(0)
#     #find mutations within
#     for mutations_df_index,mutations_df_row in mutations_df_chunk.iterrows():
            
#         if mutations_df_row.mutation_type == 'None':
#             pass
#         else:
#             #create temporary df in bed format
            
#             cols = ["chr",
#             "start",
#             "stop",
#             "mutation_type",'mutation_count']
#             temp_df = pd.DataFrame(columns=cols)
#             chr = mutations_df_row.chr
#             #if not NaN
#             if pd.notna(mutations_df_row.insertion_genomic_positions) and mutations_df_row.insertion_genomic_positions != 'nan':
#                 print(mutations_df_row)
#                 #print("Index:", index)
#                 #print(row.insertion_genomic_positions)
#                 #convert genomic positions from string to list
#                 # Convert string to list if more than one
#                 insertion_genomic_positions = mutations_df_row.insertion_genomic_positions.strip('][').split(', ')
#                 #count which mutation number currently on to be added to the temporary bed file
#                 count = 0
#                 for gen_pos in insertion_genomic_positions:
#                     count += 1
#                     #get index
#                     # if len(insertion_genomic_positions) > 1:
#                     #     index = insertion_genomic_positions.index(gen_pos)
#                     # else:
#                     #     index = 'nan'
#                     start = int(gen_pos)
#                     stop = start + 1
#                     mutation_type = "insertion"
#                     #add to temp_df
#                     temp_list = [chr,start,stop,mutation_type,count]
#                     temp_df.loc[len(temp_df)] = temp_list
#             if pd.notna(mutations_df_row.deletion_genomic_positions) and mutations_df_row.deletion_genomic_positions != 'nan':
#                 # Convert string to list
#                 deletion_genomic_positions = mutations_df_row.deletion_genomic_positions.strip('][').split(', ')
#                 #count which mutation number currently on to be added to the temporary bed file
#                 count = 0
#                 for gen_pos in deletion_genomic_positions:
#                     count += 1
#                     #get index
#                     # if len(deletion_genomic_positions) > 1:
#                     #     index = deletion_genomic_positions.index(gen_pos)
#                     # else:
#                     #     index = 'nan'
#                     start = int(gen_pos)
#                     stop = start + 1
#                     mutation_type = "deletion"
#                     #add to temp_df
#                     temp_list = [chr,start,stop,mutation_type,count]
#                     temp_df.loc[len(temp_df)] = temp_list
#             if pd.notna(mutations_df_row.substitution_genomic_positions) and mutations_df_row.substitution_genomic_positions != 'nan':
#                 # Convert string to list
#                 substitution_genomic_positions = mutations_df_row.substitution_genomic_positions.strip('][').split(', ')
#                 #count which mutation number currently on to be added to the temporary bed file
#                 count = 0
#                 for gen_pos in substitution_genomic_positions:
#                     count += 1
#                     #get index
#                     # if len(substitution_genomic_positions) > 1:
#                     #     index = substitution_genomic_positions.index(gen_pos)
#                     # else:
#                     #     index = 'nan'
#                     start = int(gen_pos)
#                     stop = start + 1
#                     mutation_type = "substitution"
#                     #add to temp_df
#                     temp_list = [chr,start,stop,mutation_type,count]
#                     temp_df.loc[len(temp_df)] = temp_list
#             #now do bedtools intersect to find which TFBSs overlap with which mutations
#             #sort by chr then start
#             temp_df = temp_df.sort_values(["chr", "start"]).reset_index(drop=True)
#             # write to buffer
#             temp_df_buffer = io.StringIO()
#             temp_df.to_csv(temp_df_buffer, sep="\t", index=False, header=None)
#             temp_df_buffer.seek(0)
            
#             output_buffer = io.StringIO()
        
#             output_buffer = merge_bedfiles(temp_df_buffer, mapped_motifs_bed, output_buffer)
            
#             #remove temp_df_buffer stream


#             #read in output buffer as df
#             output_df = pd.read_table(output_buffer, sep='\t')
            
#             #get column names and rename columns
#             output_df_cols = cols+mapped_motifs_bed_columns+['bp_overlap']
#             output_df.columns = output_df_cols
#             #for each mutation type get list of overlapping TFBSs. Add these to a dictionary column in the mutations_df_chunk
#             #create defaultdicts with lists as values so that non-existing keys can be added to in one go
#             insertion_overlapping_TFBS_family = defaultdict(list)
#             insertion_overlapping_TFBS_AGI = defaultdict(list)
#             #insertion_overlapping_TFBS_total = defaultdict(list)
#             deletion_overlapping_TFBS_family = defaultdict(list)
#             deletion_overlapping_TFBS_AGI = defaultdict(list)
#             #deletion_overlapping_TFBS_total = defaultdict(list)
#             substitution_overlapping_TFBS_family = defaultdict(list)
#             substitution_overlapping_TFBS_AGI = defaultdict(list)
#             #substitution_overlapping_TFBS_total = defaultdict(list)
            

#             for index,row in output_df.iterrows():
#                 # empty number that will increase for each insertion
#                # insertion_overlapping_TFBS_count = int()
#                #if mutation and has a 1bp overlap
#                #if no TF_AGI then pass
#                 if row.TF_AGI == '.':
#                     pass
#                 elif row.mutation_type == 'insertion' and row.bp_overlap > 0:
#                     #add insertion TFBS family information to dictionary for the correct mutation number
#                     #print(row)
#                     insertion_overlapping_TFBS_family[f'insertion{row.mutation_count}'] += [row.TF_family]
#                     #add insertion TFBS AGI information to dictionary for the correct mutation number
#                     insertion_overlapping_TFBS_AGI[f'insertion{row.mutation_count}'] += [row.TF_AGI]
#                     #add total number of TFBSs overlapping each insertion
#                  #   insertion_overlapping_TFBS_total[f'insertion{row.mutation_count}'] += 1
#                 elif row.mutation_type == 'deletion' and row.bp_overlap > 0:
#                     #add deletion TFBS family information to dictionary for the correct mutation number
#                     deletion_overlapping_TFBS_family[f'deletion{row.mutation_count}'] += [row.TF_family]
#                     #add deletion TFBS AGI information to dictionary for the correct mutation number
#                     deletion_overlapping_TFBS_AGI[f'deletion{row.mutation_count}'] += [row.TF_AGI]
#                     #add total number of TFBSs overlapping each insertion
#                    # deletion_overlapping_TFBS_total[f'deletion{row.mutation_count}'] += 1
#                 elif row.mutation_type == 'substitution' and row.bp_overlap > 0:
#                     #add substitution TFBS family information to dictionary for the correct mutation number
#                     substitution_overlapping_TFBS_family[f'substitution{row.mutation_count}'] += [row.TF_family]
#                     #add substitution TFBS AGI information to dictionary for the correct mutation number
#                     substitution_overlapping_TFBS_AGI[f'substitution{row.mutation_count}'] += [row.TF_AGI]
#                     #add total number of TFBSs overlapping each insertion
#                  #   substitution_overlapping_TFBS_total[f'substitution{row.mutation_count}'] += 1
#             # #calculate total unique TFBS for each insertion, deletion and subsitution
#             # insertion_overlapping_TFBS_total_unique = []
#             # for insertion,AGI in insertion_overlapping_TFBS_AGI.items():
#             #     insertion_overlapping_TFBS_total_unique += np.unique(AGI).astype(list)
#             # deletion_overlapping_TFBS_total_unique = []
#             # for deletion,AGI in deletion_overlapping_TFBS_AGI.items():
#             #     print(f'AGIunique={np.unique(AGI).astype(list)}')
#             #     print(f'total={deletion_overlapping_TFBS_total_unique}')
#             #     print(f'AGI={AGI}')
#             #     deletion_overlapping_TFBS_total_unique += np.unique(AGI).astype(list)
#             #     print(f'newtotal={deletion_overlapping_TFBS_total_unique}')
#             # substitution_overlapping_TFBS_total_unique = []
#             # for substitution,AGI in substitution_overlapping_TFBS_AGI.items():
#             #     substitution_overlapping_TFBS_total_unique += np.unique(AGI).astype(list)
                
#             #add values to mutations_df row
#             #print(f'insertion_overlapping_TFBS_family={dict(insertion_overlapping_TFBS_family)}')
#             #print(mutations_df_row)
#             #first make overlapping TFBS families and AGIs unique
#             insertion_overlapping_TFBS_family = dict(insertion_overlapping_TFBS_family)
#             insertion_overlapping_TFBS_AGI = dict(insertion_overlapping_TFBS_AGI)
#             deletion_overlapping_TFBS_family = dict(deletion_overlapping_TFBS_family)
#             deletion_overlapping_TFBS_AGI = dict(deletion_overlapping_TFBS_AGI)
#             substitution_overlapping_TFBS_family = dict(substitution_overlapping_TFBS_family)
#             substitution_overlapping_TFBS_AGI = dict(substitution_overlapping_TFBS_AGI)
#             #if empty dictionary, change to nan
#             if insertion_overlapping_TFBS_family == {}:
#                 insertion_overlapping_TFBS_family = np.nan
#             else:
#                 #keep only unique TFBS families
#                 for k,v in insertion_overlapping_TFBS_family.items():
#                     #print(np.unique(v).astype(list))
#                     insertion_overlapping_TFBS_family[k] = np.unique(v).tolist()

#             if insertion_overlapping_TFBS_AGI == {}:
#                 insertion_overlapping_TFBS_AGI = np.nan
#             else:
#                 #keep only unique TFBS AGIs
#                 for k,v in insertion_overlapping_TFBS_AGI.items():
#                     #print(np.unique(v).astype(list))
#                     insertion_overlapping_TFBS_AGI[k] = np.unique(v).tolist()

#             if deletion_overlapping_TFBS_family == {}:
#                 deletion_overlapping_TFBS_family = 'nan'
#             else:
#                 #keep only unique TFBS families
#                 for k,v in deletion_overlapping_TFBS_family.items():
#                     #print(np.unique(v).astype(list))
#                     deletion_overlapping_TFBS_family[k] = np.unique(v).tolist()

#             if deletion_overlapping_TFBS_AGI == {}:
#                 deletion_overlapping_TFBS_AGI = np.nan
#             else:
#                 #keep only unique TFBS AGIs
#                 for k,v in deletion_overlapping_TFBS_AGI.items():
#                     #print(np.unique(v).astype(list))
#                     deletion_overlapping_TFBS_AGI[k] = np.unique(v).tolist()
            
#             if substitution_overlapping_TFBS_family == {}:
#                 substitution_overlapping_TFBS_family = np.nan
#             else:
#                 #keep only unique TFBS families
#                 for k,v in substitution_overlapping_TFBS_family.items():
#                     #print(np.unique(v).astype(list))
#                     substitution_overlapping_TFBS_family[k] = np.unique(v).tolist()
                    
#             if substitution_overlapping_TFBS_AGI == {}:
#                 substitution_overlapping_TFBS_AGI = np.nan
#             else:
#                 #keep only unique TFBS AGIs
#                 for k,v in substitution_overlapping_TFBS_AGI.items():
#                     #print(np.unique(v).astype(list))
#                     substitution_overlapping_TFBS_AGI[k] = np.unique(v).tolist()


#             # def write_values(mutations_df_chunk, mutations_df_index, col_name, value):
#             #     """function to map_partitions with dask to allow loc"""
#             #     #make copy of mutations_df_chunk
#             #     #mutations_df_chunk = mutations_df_chunk.copy()
#             #     #df.at can only access a single value at a time.
#             #     #df.loc can select multiple rows and/or columns.
#             #     #mutations_df_chunk.loc[mutations_df_index, col_name] = str(value)
#             #     #find correct row from index
#             #     mutations_df_chunk[col_name] = mutations_df_chunk[col_name].mask(mutations_df_chunk[mutations_df_chunk.index==mutations_df_index],str(value))
#             #     return mutations_df_chunk
#             #write values
#             # mutations_df_chunk = write_values(mutations_df_chunk, mutations_df_index, 'insertion_overlapping_TFBS_family', insertion_overlapping_TFBS_family)
#             # mutations_df_chunk = write_values(mutations_df_chunk, mutations_df_index, 'insertion_overlapping_TFBS_AGI', insertion_overlapping_TFBS_AGI)
#             # mutations_df_chunk = write_values(mutations_df_chunk, mutations_df_index, 'deletion_overlapping_TFBS_family', deletion_overlapping_TFBS_family)
#             # mutations_df_chunk = write_values(mutations_df_chunk, mutations_df_index, 'deletion_overlapping_TFBS_AGI', deletion_overlapping_TFBS_AGI)
#             # mutations_df_chunk = write_values(mutations_df_chunk, mutations_df_index, 'substitution_overlapping_TFBS_family', substitution_overlapping_TFBS_family)
#             # mutations_df_chunk = write_values(mutations_df_chunk, mutations_df_index, 'substitution_overlapping_TFBS_AGI', substitution_overlapping_TFBS_AGI)
#             mutations_df_chunk["insertion_overlapping_TFBS_family"] = mutations_df_chunk["insertion_overlapping_TFBS_family"].mask(mutations_df_chunk[mutations_df_chunk['index_col']==mutations_df_index],str(insertion_overlapping_TFBS_family))
#             #make copy of mutations_df_chunk
#             #mutations_df_chunk.loc[mutations_df_index, "insertion_overlapping_TFBS_family"] = str(insertion_overlapping_TFBS_family)
#            # print(mutations_df_chunk.query('column in @mutations_df_index'))
#            # mutations_df_chunk["insertion_overlapping_TFBS_family"] = mutations_df_chunk["insertion_overlapping_TFBS_family"].mask(mutations_df_chunk.query('@mutations_df_index'),str(insertion_overlapping_TFBS_family))
#             #mutations_df_chunk.loc[mutations_df_index, "insertion_overlapping_TFBS_AGI"]= str(insertion_overlapping_TFBS_AGI)
#             #mutations_df_chunk["insertion_overlapping_TFBS_AGI"] = mutations_df_chunk["insertion_overlapping_TFBS_AGI"].mask(mutations_df_chunk.query('@mutations_df_index'),str(insertion_overlapping_TFBS_AGI))
#             #mutations_df_chunk.loc[mutations_df_index, "deletion_overlapping_TFBS_family"]= str(deletion_overlapping_TFBS_family)
#             #mutations_df_chunk["deletion_overlapping_TFBS_family"] = mutations_df_chunk["deletion_overlapping_TFBS_family"].mask(mutations_df_chunk.query('@mutations_df_index'),str(deletion_overlapping_TFBS_family))
#             #mutations_df_chunk.loc[mutations_df_index, "deletion_overlapping_TFBS_AGI"]= str(deletion_overlapping_TFBS_AGI)
#             #mutations_df_chunk["deletion_overlapping_TFBS_AGI"] = mutations_df_chunk["deletion_overlapping_TFBS_AGI"].mask(mutations_df_chunk.query('@mutations_df_index'),str(deletion_overlapping_TFBS_AGI))
#           #  mutations_df_chunk.loc[mutations_df_index, "substitution_overlapping_TFBS_family"]= str(substitution_overlapping_TFBS_family)
#             #mutations_df_chunk["substitution_overlapping_TFBS_family"] = mutations_df_chunk["substitution_overlapping_TFBS_family"].mask(mutations_df_chunk.query('@mutations_df_index'),str(substitution_overlapping_TFBS_family))
#            # mutations_df_chunk.loc[mutations_df_index, "substitution_overlapping_TFBS_AGI"]= str(substitution_overlapping_TFBS_AGI)
#             #mutations_df_chunk["substitution_overlapping_TFBS_AGI"] = mutations_df_chunk["substitution_overlapping_TFBS_AGI"].mask(mutations_df_chunk.query('@mutations_df_index'),str(substitution_overlapping_TFBS_AGI))
#             # row.insertion_overlapping_TFBS_total_unique= insertion_overlapping_TFBS_total_unique
#             # row.deletion_overlapping_TFBS_total_unique= deletion_overlapping_TFBS_total_unique
#             # row.substitution_overlapping_TFBS_total_unique= substitution_overlapping_TFBS_total_unique

#             #go back to start of buffer
#             mapped_motifs_bed.seek(0)

#     return mutations_df_chunk

In [33]:
def chunkify(mutations_df,mapped_motifs_df,output_folder,gene):
    """function to prepare dfs and slice mutations_df into chunks to reduce memory before running find_overlapping_TFBSs() """
   

    #for each guide containing mutations, create a temporary bed file containing each mutation and then do bedtools intersect to find which overlap TFBs
    #then add the TFBS names into a new column for that row

    #get column names from mapped_motifs_df
    mapped_motifs_bed_columns = list(mapped_motifs_df.columns)


    #turn mapped_motifs_df into a buffer
    mapped_motifs_bed = io.StringIO()
    mapped_motifs_df.to_csv(mapped_motifs_bed, sep="\t", index=False, header=None)
    #go back to start of buffer
    mapped_motifs_bed.seek(0)

    #add columns to mutations_df
    new_columns = ['insertion_overlapping_TFBS_family',
    'insertion_overlapping_TFBS_AGI',
    'deletion_overlapping_TFBS_family',
    'deletion_overlapping_TFBS_AGI',
    'substitution_overlapping_TFBS_family',
    'substitution_overlapping_TFBS_AGI',
                  ]
    #first make a new df that will merge into mutations_df
    temp_new_df = pd.DataFrame(columns=new_columns)
    mutations_df = pd.concat([mutations_df,temp_new_df], axis=1)
    #print(mutations_df)

    #first make certain columns string
    #make columns containing lists string for now so can use groupby
    to_string = ['insertion_positions','deletion_positions','substitution_positions','insertion_cut_site_distance','deletion_cut_site_distance','substitution_cut_site_distance','insertion_positions_relative_to_TSS','insertion_genomic_positions','deletion_positions_relative_to_TSS','deletion_genomic_positions','substitution_positions_relative_to_TSS','substitution_genomic_positions']
    mutations_df[to_string] = mutations_df[to_string].astype(str)


    # chunksize = 5
    # def df_chunking(df,mapped_motifs_bed,mapped_motifs_bed_columns, chunksize):
    #     """Splits df into chunks, drops data of original df inplace"""
    #     count = 0 # Counter for chunks
    #     #create list of chunks
    #     chunks = []
    #     while len(df):
    #         count += 1
    #         print('Preparing chunk {}'.format(count))
    #         # Return df chunk
    #         chunk = df.iloc[:chunksize].copy()
    #         #go back to start of buffer
    #         mapped_motifs_bed.seek(0)
    #         new_chunk = find_overlapping_TFBSs(chunk,mapped_motifs_bed,mapped_motifs_bed_columns)
    #         #append chunk to list
    #         chunks.append(new_chunk)
    #         # Delete data in place because it is no longer needed
    #         df.drop(df.index[:chunksize], inplace=True)
    #     return chunks

    # chunks = df_chunking(mutations_df,mapped_motifs_bed,mapped_motifs_bed_columns, chunksize)
  
#     # Job parameters
#     n_jobs = 4  # Poolsize
#     chunksize = 5  # Maximum size of Frame Chunk
#     # Preparation
#    # df = pd.DataFrame(np.random.rand(*size))
#     ctx = mp.get_context('spawn')
#     pool = ctx.Pool(n_jobs)
#     #pool = mp.Pool(n_jobs)
#     print('Starting MP')
#     # Execute the wait and print function in parallel
#     #create list of chunks
#     # chunks = []
#     #mutations_df = pd.concat(pool.imap(find_overlapping_TFBSs(df,mapped_motifs_bed,mapped_motifs_bed_columns), df_chunking(df, chunksize)))
#     # mutations_df = pd.concat(pool.starmap(lambda mutations_df: find_overlapping_TFBSs(mutations_df,mapped_motifs_bed,mapped_motifs_bed_columns), df_chunking(mutations_df, chunksize)))
#     #mutations_df = pd.concat(pool.starmap(find_overlapping_TFBSs,[(mutations_df,mapped_motifs_bed,mapped_motifs_bed_columns)], df_chunking(mutations_df, chunksize)))
#     mutations_df = pd.concat(pool.starmap(df_chunking,[(mutations_df, chunksize)]))
#     # p.starmap(print_name, [('Thomas', 'Scott'), ('Ali', 'Khan')])
#     #concatenate the chunks
#    # mutations_df = pd.concat(p.map(Simulation, range(10))
#     pool.close()
#     pool.join()
#     print('DONE')

    #convert mutations_df into chunks to reduce memory load
    CHUNK_SIZE = 5

    index_slices = sliced(range(len(mutations_df)), CHUNK_SIZE)
    #create list of chunks
    chunks = []
    for index_slice in index_slices:
        #go back to start of buffer
        mapped_motifs_bed.seek(0)
        chunk = mutations_df.iloc[index_slice] # your dataframe chunk ready for use
        new_chunk = find_overlapping_TFBSs(chunk,mapped_motifs_bed,mapped_motifs_bed_columns)

        
        chunks.append(new_chunk)

    


            

    #concatenate chunks into mutations_df
    mutations_df = pd.concat(chunks)            
    #write out mutations_df
    mutations_df.to_csv(f'{output_folder}{gene}_TFBSoverlapping.tsv', sep="\t", index=False, header=1)
                     
    return mutations_df



In [34]:
# def chunkify(mutations_df,mapped_motifs_df,output_folder,gene,threads):
#     """function to prepare dfs and slice mutations_df into chunks to reduce memory before running find_overlapping_TFBSs() """
   

#     #for each guide containing mutations, create a temporary bed file containing each mutation and then do bedtools intersect to find which overlap TFBs
#     #then add the TFBS names into a new column for that row

#     #get column names from mapped_motifs_df
#     mapped_motifs_bed_columns = list(mapped_motifs_df.columns)


#     #turn mapped_motifs_df into a buffer
#     mapped_motifs_bed = io.StringIO()
#     mapped_motifs_df.to_csv(mapped_motifs_bed, sep="\t", index=False, header=None)
#     #go back to start of buffer
#     mapped_motifs_bed.seek(0)

#     #add columns to mutations_df
#     new_columns = ['insertion_overlapping_TFBS_family',
#     'insertion_overlapping_TFBS_AGI',
#     'deletion_overlapping_TFBS_family',
#     'deletion_overlapping_TFBS_AGI',
#     'substitution_overlapping_TFBS_family',
#     'substitution_overlapping_TFBS_AGI',
#                   ]
#     #first make a new df that will merge into mutations_df
#     temp_new_df = pd.DataFrame(columns=new_columns)
#     mutations_df = pd.concat([mutations_df,temp_new_df], axis=1)
#     #print(mutations_df)

#     #first make certain columns string
#     #make columns containing lists string for now so can use groupby
#     to_string = ['insertion_positions','deletion_positions','substitution_positions','insertion_cut_site_distance','deletion_cut_site_distance','substitution_cut_site_distance','insertion_positions_relative_to_TSS','insertion_genomic_positions','deletion_positions_relative_to_TSS','deletion_genomic_positions','substitution_positions_relative_to_TSS','substitution_genomic_positions']
#     mutations_df[to_string] = mutations_df[to_string].astype(str)
#     #add index column for dask
#     mutations_df['index_col']=mutations_df.index
#     #save mutations_df as a temporary tsv
#     mutations_df.to_csv(f'{output_folder}{gene}_temp.tsv', sep="\t", index=False, header=1)


#     # Load the data with Dask instead of Pandas to allow chunking and multithreading.
#     dask_df = dd.read_table(
#         f'{output_folder}{gene}_temp.tsv', sep="\t",header=0,
#         blocksize=5 * 1024 * 1024, # 5MB chunks
        
#     )

#     # Setup the calculation graph; unlike Pandas code,
#     # no work is done at this point:

#     mutations_df = find_overlapping_TFBSs(dask_df,mapped_motifs_bed,mapped_motifs_bed_columns)

#     # Actually run the computation, using n threads:
#     #mutations_df = dd.map_partitions(lambda dask_df: find_overlapping_TFBSs(dask_df,mapped_motifs_bed,mapped_motifs_bed_columns)).compute(num_workers=threads)
#     mutations_df = mutations_df.compute(num_workers=threads)
#     # Sort using normal Pandas DataFrame, since Dask's Pandas emulation doesn't implement this method:
#     mutations_df.sort_values(['platename','plant_ID','guide_number'], ascending=True, inplace = True)
           
#     #write out mutations_df
#     mutations_df.to_csv(f'{output_folder}{gene}_TFBSoverlapping.tsv', sep="\t", index=False, header=1)
                     
#     return mutations_df      



In [35]:
#written in dask format rather than pandas
def genotype_plant_lines(mutations_df,output_folder,gene):
    """function to decide whether each plant line is -homozygous 
    -biallelic - each mutated at same location/site twice
    -chimeric - different mutations in different cells or tissues - dont analyse if chimeric - 1 small region, if more than 2 mutations then probably chimeric - ie.    probably still has tDNA
    Multiple guide sites - eg. multiple agro strains in each plant. Check for paired guides and whether guides that aren’t meant to be paired are paired.
    """
    #homozygouslines are those with no duplicated guides in each plant line
    mutations_df.loc[~mutations_df.duplicated(['plant_ID','guide']),'genotype'] = 'homozygous'
    #mutations_df['genotype'] = mutations_df['genotype'].mask(~mutations_df.duplicated(['plant_ID','guide']), 'homozygous')
    #biallelic
    #N = 2
    #mutations_df_copy = mutations_df[mutations_df.duplicated(['plant_ID','guide']) | mutations_df['count'].ge(N)]
    #if 70% of reads or more are the same then mark as homozygous
    mutations_df.loc[mutations_df.read_percentage >= 70, 'genotype'] = 'homozygous'
    #if 10% of reads or less are that mutation, mark genotype as 'nan
    mutations_df.loc[mutations_df.read_percentage <=10, 'genotype'] = 'nan'
    #if between 10 and 70% of reads, mark genotype as heterozygous
    mutations_df.loc[(mutations_df.read_percentage > 10) & (mutations_df.read_percentage < 70), 'genotype'] = 'heterozygous'

    #create a count column
    mutations_df['number_of_different_alleles'] = int()
    #create non wild type count column
    mutations_df['number_of_different_non_wt_alleles'] = int()
    
    #create a sum_of_count column
    #mutations_df['sum_of_count'] = int()
    #count how many duplicates there are for heterozygous lines.
    mutations_df.loc[mutations_df.genotype == 'heterozygous','number_of_different_alleles'] = mutations_df[mutations_df.genotype == 'heterozygous'].groupby(['plant_ID','guide'])['number_of_different_alleles'].transform('count')
    #make non wild type count
    mutations_df.loc[(mutations_df.genotype == 'heterozygous')&~(mutations_df.mutation_type == 'None'),'number_of_different_non_wt_alleles'] = mutations_df[(mutations_df.genotype == 'heterozygous')&~(mutations_df.mutation_type == 'None')].groupby(['plant_ID','guide'])['number_of_different_non_wt_alleles'].transform('count')
    #make count numeric
    #mutations_df['count'] = mutations_df['count'].astype(int)
    #print(mutations_df.dtypes)
    #if heterozygous count is 1 then homozygous
    mutations_df.loc[(mutations_df.genotype == 'heterozygous') & (mutations_df.number_of_different_alleles == 1),'genotype'] = 'homozygous'
    #print(mutations_df.loc[(mutations_df.genotype == 'heterozygous') & (mutations_df.count == 1)])
    #
    #if homozygous then count is 1
    mutations_df.loc[mutations_df.genotype == 'homozygous','number_of_different_alleles'] = 1

    #if mutation_type is None, count is 0
    #create mask for wildtype
    # wt_mask = (mutations_df['mutation_type']=='None')
    # mutations_df.loc[wt_mask, 'count'] = 0

    #get the sum of the counts for each set of duplicates( wild type is 0, each mutation is counted as 1)
    # sort=False, as_index=False

   # mutations_df['sum_of_count'] = mutations_df.groupby(['plant_ID','guide'])['sum_of_count'].agg({"count":"sum"})
    #print(mutations_df.groupby(['plant_ID','guide'],sort=False, as_index=False)['count'].agg({"sum_of_count":"sum"}))
   # print(mutations_df.groupby(['plant_ID','guide'])['count'].transform('count'))



    #if number_of_different_alleles is two and no wild type is present in either of the groups of reads then biallelic
    mutations_df.loc[(mutations_df['number_of_different_non_wt_alleles']==2)&~(mutations_df.mutation_type == 'None'),'genotype'] = 'biallelic'
    
    #if number_of_different_alleles more than two then chimeric
    mutations_df.loc[mutations_df['number_of_different_alleles']>2,'genotype'] = 'chimeric'

    # reset count column
    #filtered['count'] = filtered.groupby(['plant_ID','guide'])[['plant_ID','guide']].transform('count')

    #df1.loc[df['count'] < N, 'count'] = 1
    #mutations_df[mutations_df.duplicated('plant_ID','guide')]
    #add plant
    #print(mutations_df_copy)
    #change column order
    mutations_df = mutations_df[['chr', 'plant_ID', 'platename', 'library', 'first_reaction_primers',
       'second_reaction_primers', 'guide', 'guide_number', 'aligned_sequence',
       'reference_sequence', 'mutation_type','genotype', 'read_number', 'read_percentage',
       'insertion_positions', 'deletion_positions', 'substitution_positions',
       'insertion_cut_site_distance', 'deletion_cut_site_distance',
       'substitution_cut_site_distance', 'cut_site_promoter_position',
       'insertion_positions_relative_to_TSS', 'insertion_genomic_positions',
       'deletion_positions_relative_to_TSS', 'deletion_genomic_positions',
       'substitution_positions_relative_to_TSS',
       'substitution_genomic_positions', 'insertion_overlapping_TFBS_family',
       'insertion_overlapping_TFBS_AGI', 'deletion_overlapping_TFBS_family',
       'deletion_overlapping_TFBS_AGI', 'substitution_overlapping_TFBS_family',
       'substitution_overlapping_TFBS_AGI','number_of_different_alleles','number_of_different_non_wt_alleles'
       ]]
    #remove genotype 'nan'
    mutations_df = mutations_df[~(mutations_df.genotype == 'nan')]
    
   
   
    #save df
    mutations_df.to_csv(f'{output_folder}{gene}_TFBSoverlapping_categorised.tsv', sep="\t", index=False, header=1)

    return mutations_df
    



In [61]:
def categorise_guide_pairs(mutations_df_genotyped, guide_pairs_df):
    """function to check which guide pairs where delivered to each plant line and to check whether mutations at more than one guide site are within the other guide pair"""
    #first get unique guides based on 2 columns in guide_pairs_df    
    unique_guides = pd.concat([guide_pairs_df['guide1'],guide_pairs_df['guide2']]).unique()
    #for guides in unique guides list, add dictionary value of all other potential guides that it is paired with
    #create defaultdict with lists as values so that non-existing keys can be added to in one go
    guide_dict = defaultdict(list)
    for guide in unique_guides:
        #check for instances of that guide in the first column
        filtered_col_1 = guide_pairs_df[guide_pairs_df.guide1==guide]['guide2'].to_list()
        #then do the same for the second column
        filtered_col_2 = guide_pairs_df[guide_pairs_df.guide2==guide]['guide1'].to_list()
        #create list of unique values
        list_of_unique = np.unique((filtered_col_1 + filtered_col_2)).astype(list)
        #append each value in list to dict
        for val in list_of_unique:
            guide_dict[guide] += [val]
        
        
        
    #turn into normal dict
    guide_dict = dict(guide_dict)

    #mutations_df_genotyped df, filter out non mutated guides. Then create a dictionary of guides which were mutated. Then compare the guide pairs in guide_dict to the mutated guides.
    #make a new df with one plant line per row, detailing how many guides sites were mutated, which guide sites where mutated and then if more than 1 guide site is mutated, if more than 3 guide pairs then flag that more than one agro strain and tDNA was inserted. If 2 mutated guide sites, flag whether the guides were guide pairs or whether the mutations were at sites not in a pair. If not in a pair then flag that plant line as having more than one agro strain and tDNA was inserted
    #add any additional metrics for overall lines
    #finally, merge this df with the mutations_df_genotyped (applying the same value in each column within each plant line)

    

In [38]:
mapped_motifs_bed = '../../data/output/non-overlapping_includingbidirectional_all_genes_newannotation_3KB/FIMO/promoters_5UTR_motifs_mapped_q0_05.bed'
mutations_ARF9 = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/ARF9_merged.tsv'
mutations_ARF18 = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/ARF18_merged.tsv'
mutations_DREB26 = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/DREB26_merged.tsv'
mutations_NLP7 = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/NLP7_merged.tsv'
output_folder = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/Variant_call/'
guide_pairs = f'../../data/CRISPR_library/pacbio/demultiplexed/Data_Package_Batch_04_04_2022/Sam_Witham_EI_SW_ENQ-5142_A_01_Additional_Barcode_Analysis/References/all_guide_pairs.csv'


In [39]:
#read in files
mutations_ARF9_df,mapped_motifs_df,guide_pairs_df =  read_in_files(mutations_ARF9, mapped_motifs_bed, guide_pairs)

In [40]:
mutations_ARF9_df

,chr,plant_ID,platename,library,first_reaction_primers,second_reaction_primers,guide,guide_number,aligned_sequence,reference_sequence,...,insertion_cut_site_distance,deletion_cut_site_distance,substitution_cut_site_distance,cut_site_promoter_position,insertion_positions_relative_to_TSS,insertion_genomic_positions,deletion_positions_relative_to_TSS,deletion_genomic_positions,substitution_positions_relative_to_TSS,substitution_genomic_positions
0,4,plntEPSWT20057-1,p1ARF9,1017,SW346_SW442,ARF9_1F_8R,ARF9_guide1,1,TAGGTTACAGTTACAGAGCAGGAAGGATTGCGTTGCGTTG,GAGGTTACAGTTACAGAGCAGGAAGGATTGCGTTGCGTTA,...,NaN,NaN,"[-20, 19]",1236,NaN,NaN,NaN,NaN,"[198, 237]","[12451340, 12451379]"
1,4,plntEPSWT20057-1,p1ARF9,1017,SW346_SW442,ARF9_1F_8R,ARF9_guide1,1,GAGGTTACAGTTACAGAGCAGGAAGGATTGCGTTGCGTTA,GAGGTTACAGTTACAGAGCAGGAAGGATTGCGTTGCGTTA,...,NaN,NaN,NaN,1236,NaN,NaN,NaN,NaN,NaN,NaN
2,4,plntEPSWT20057-1,p1ARF9,1017,SW346_SW442,ARF9_1F_8R,ARF9_guide2,2,TAGAGAGGTTACAGTTACAGAGCAGGAAGGATTGCGTTG-,TAGAGAGGTTACAGTTACAGAGCAGGAAGGATTGCGTTGC,...,NaN,[19],NaN,1232,NaN,NaN,[233],[12451375],NaN,NaN
3,4,plntEPSWT20057-1,p1ARF9,1017,SW346_SW442,ARF9_1F_8R,ARF9_guide2,2,TAGAGAGGTTACAGTTACAGAGCAGGAAGGATTGCGTTGC,TAGAGAGGTTACAGTTACAGAGCAGGAAGGATTGCGTTGC,...,NaN,NaN,NaN,1232,NaN,NaN,NaN,NaN,NaN,NaN
4,4,plntEPSWT20057-1,p1ARF9,1017,SW346_SW442,ARF9_1F_8R,ARF9_guide3,3,-TTTTACGGGAGAGAAATTAGAGAGGTTACAGTTACAGAG,GTTTTACGGGAGAGAAATTAGAGAGGTTACAGTTACAGAG,...,NaN,"[-21, -20]",NaN,1214,NaN,NaN,"[175, 176]","[12451317, 12451318]",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401,4,plntEPSWT20067-8,p1ARF9,1017,SW357_SW453,ARF9_1F_5R,ARF9_guide28,28,TAATCCAAGTCTAGCCAGTATTGTTGCCTGCATAAGAGCA,TAATCCAAGTCTAGCCAGTATTGTTGCCTGCATAAGAGCA,...,NaN,NaN,NaN,230,NaN,NaN,NaN,NaN,NaN,NaN
402,4,plntEPSWT20067-8,p1ARF9,1017,SW357_SW453,ARF9_1F_5R,ARF9_guide29,29,GCTTCGTCTCTAATCCAAGTCTAGCCAGTATTGTTGCCTG,GCTTCGTCTCTAATCCAAGTCTAGCCAGTATTGTTGCCTG,...,NaN,NaN,NaN,220,NaN,NaN,NaN,NaN,NaN,NaN
403,4,plntEPSWT20067-8,p1ARF9,1017,SW357_SW453,ARF9_1F_8R,ARF9_guide29,29,GCTTCGTCTCTAATCCAAGTCTAGCCAGTATTGTTGCCTG,GCTTCGTCTCTAATCCAAGTCTAGCCAGTATTGTTGCCTG,...,NaN,NaN,NaN,220,NaN,NaN,NaN,NaN,NaN,NaN
404,4,plntEPSWT20067-8,p1ARF9,1017,SW357_SW453,ARF9_1F_8R,ARF9_guide30,30,AAATGTAATTTATCCAAAGAAGGAGGTCAAAGCTGCTTCG,AAATGTAATTTATCCAAAGAAGGAGGTCAAAGCTGCTTCG,...,NaN,NaN,NaN,186,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
# mutations_df = find_overlapping_TFBSs(mutations_ARF9_df,mapped_motifs_df,output_folder,'ARF9')
mutations_df = chunkify(mutations_ARF9_df,mapped_motifs_df,output_folder,'ARF9')

In [44]:
mutations_df_genotyped = genotype_plant_lines(mutations_df,output_folder,'ARF9')

In [62]:
categorise_guide_pairs(mutations_df_genotyped,guide_pairs_df)

{'ARF9_guide1': ['ARF9_guide5'], 'ARF9_guide2': ['ARF9_guide5'], 'ARF9_guide3': ['ARF9_guide5'], 'ARF9_guide4': ['ARF9_guide6'], 'ARF9_guide11': ['ARF9_guide6', 'ARF9_guide7', 'ARF9_guide8'], 'ARF9_guide9': ['ARF9_guide12'], 'ARF9_guide10': ['ARF9_guide12', 'ARF9_guide13'], 'ARF9_guide14': ['ARF9_guide13'], 'ARF9_guide15': ['ARF9_guide23'], 'ARF9_guide16': ['ARF9_guide23'], 'ARF9_guide17': ['ARF9_guide23'], 'ARF9_guide30': ['ARF9_guide23', 'ARF9_guide24'], 'ARF9_guide18': ['ARF9_guide25'], 'ARF9_guide19': ['ARF9_guide25'], 'ARF9_guide20': ['ARF9_guide26'], 'ARF9_guide21': ['ARF9_guide26', 'ARF9_guide27'], 'ARF9_guide22': ['ARF9_guide28', 'ARF9_guide29'], 'ARF18_guide1': ['ARF18_guide5'], 'ARF18_guide2': ['ARF18_guide5'], 'ARF18_guide12': ['ARF18_guide5', 'ARF18_guide6'], 'ARF18_guide3': ['ARF18_guide7'], 'ARF18_guide14': ['ARF18_guide19', 'ARF18_guide20', 'ARF18_guide21', 'ARF18_guide7'], 'ARF18_guide4': ['ARF18_guide9'], 'ARF18_guide17': ['ARF18_guide10', 'ARF18_guide9'], 'ARF18_guide